<a href="https://colab.research.google.com/github/shpotes/tensorflowers/blob/entregable/notebooks/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd /content/
!rm -rf tensorflowers
!git clone -b entregable https://github.com/shpotes/tensorflowers.git
%cd tensorflowers
!pip install -qq -r requirements.txt

/content
Cloning into 'tensorflowers'...
remote: Enumerating objects: 426, done.
remote: Counting objects: 100% (426/426), done.
remote: Compressing objects: 100% (285/285), done.
remote: Total 426 (delta 212), reused 306 (delta 117), pack-reused 0
Receiving objects: 100% (426/426), 126.03 MiB | 30.11 MiB/s, done.
Resolving deltas: 100% (212/212), done.
/content/tensorflowers


In [1]:
%cd /content/tensorflowers/
!git pull

/content/tensorflowers
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 5 (delta 4), reused 5 (delta 4), pack-reused 0
Unpacking objects: 100% (5/5), done.
From https://github.com/shpotes/tensorflowers
   f619003..5e373ec  entregable -> origin/entregable
Updating f619003..5e373ec
Fast-forward
 src/data/pl_datamodule.py | 32 ++++++++++++++------------------
 1 file changed, 14 insertions(+), 18 deletions(-)


In [2]:
from functools import partial

import timm
from timm.data.constants import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD
import numpy as np
import torch
import torch.nn as nn
from torchvision import models
from torchvision import transforms as T
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint

from src.data import TFColDataModule, create_train_transformations
from src.model import HydraModule
from src.utils.training_utils import turn_off_bn, load_backbone

Using custom data configuration default


In [3]:
dm = TFColDataModule(
    image_train_transforms=create_train_transformations(with_rand_augmentation=False),
    image_eval_transforms=T.Compose([
      T.Resize(224),
      T.ToTensor(),
      T.Normalize(IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD)
    ]),
    batch_size=64,
)

In [7]:
backbone = load_backbone(
    model_name="resnet50d",
    checkpoint="https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-rsb-weights/resnet50d_a1_0-e20cff14.pth",
)
turn_off_bn(backbone)

model = HydraModule(
    backbone,
    lr=3e-4,
    clf_loss="asl"
)

In [8]:
!wandb login

wandb: Currently logged in as: shpotes (use `wandb login --relogin` to force relogin)


In [9]:
logger = WandbLogger(
    project="challenge", 
    name="last resort",
    entity="tensorflowers",
)
trainer = pl.Trainer(
    max_epochs=50,
    gpus=1,
    logger=logger    
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [ ]:
trainer.fit(model, dm)

Using custom data configuration default


  0%|          | 0/3 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/687M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/469k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/225M [00:00<?, ?B/s]

...

Validating: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/functional.py:405: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "
/usr/local/lib/python3.7/dist-packages/torchvision/transforms/functional.py:405: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


Validating: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/functional.py:405: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "
/usr/local/lib/python3.7/dist-packages/torchvision/transforms/functional.py:405: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


Validating: 0it [00:00, ?it/s]